# 3D Point Cloud Classification with Simple PointNet

In [ ]:
!pip install plyfile

In [ ]:
!pip install open3d

Normalization

In [ ]:
def normalize_point_cloud(points):
    """
    Normalize a point cloud to be centered at the origin and fit within a unit sphere.

    :param points: (N, 3) numpy array representing the point cloud.
    :return: Normalized point cloud.
    """
    # 1. Centering: Shift the centroid to the origin
    centroid = np.mean(points, axis=0)
    points -= centroid

    # 2. Scaling: Normalize by the max distance to the origin
    max_distance = np.max(np.linalg.norm(points, axis=1))
    points /= max_distance

    return points, centroid, max_distance

Denormalization

In [ ]:
def denormalize_point_cloud(points, centroid, max_distance):
    """
    Denormalize a point cloud to its original coordinates.

    :param points: Normalized point cloud.
    :param centroid: Centroid of the original point cloud.
    :param max_distance: Maximum distance from the origin.
    :return: Denormalized point cloud.
    """
    # 1. Reverse scaling
    points *= max_distance

    # 2. Reverse centering
    points += centroid

    return points

Loader Function

In [ ]:
# loader function
import open3d as o3d
import numpy as np
from plyfile import PlyData
from sklearn.neighbors import NearestNeighbors

def load_and_prepare_data(ply_path, num_points=10000):
    # Load the PLY file
    ply_data = PlyData.read(ply_path)

    # Extract vertices and labels
    vertices = np.vstack([
        ply_data['vertex']['x'],
        ply_data['vertex']['y'],
        ply_data['vertex']['z']
    ]).T  # Shape: (N, 3)

    vertex_labels = np.array(ply_data['vertex']['label'])  # Shape: (N,)

    # Load the mesh as an Open3D object
    mesh = o3d.io.read_triangle_mesh(ply_path)

    # Sample points from the mesh
    point_cloud = mesh.sample_points_uniformly(number_of_points=num_points)

    # Extract point cloud coordinates
    points = np.asarray(point_cloud.points)

    # Find nearest neighbors between point cloud and mesh vertices
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(vertices)
    distances, indices = nbrs.kneighbors(points)

    # Assign labels to the point cloud
    point_labels = vertex_labels[indices.flatten()]

    return points, point_labels


Network Architecture

In [ ]:
# network architecture
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimplePointNet(nn.Module):
    def __init__(self, num_classes):
        super(SimplePointNet, self).__init__()
        self.num_classes = num_classes

        # Shared MLP for feature extraction
        self.mlp1 = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )

        # Fully connected layers for per-point classification
        self.mlp2 = nn.Sequential(
            nn.Conv1d(1024, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Conv1d(512, 256, 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, num_classes, 1)  # Output per point
        )

    def forward(self, x):
        # Input shape: (batch_size, 3, num_points)
        x = self.mlp1(x)  # Shape: (batch_size, 1024, num_points)
        x = self.mlp2(x)  # Shape: (batch_size, num_classes, num_points)

        return x  # Shape: (batch_size, num_classes, num_points)


Load data

In [ ]:
pwd

In [ ]:
# load point cloud data
from torch.utils.data import Dataset, DataLoader

class PointCloudDataset(Dataset):
    def __init__(self, file_paths, num_points=10000):
        self.file_paths = file_paths
        self.num_points = num_points

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        ply_path = self.file_paths[idx]
        points, labels = load_and_prepare_data(ply_path, self.num_points)
        points, centroid, max_distance = normalize_point_cloud(points)
        return torch.tensor(points, dtype=torch.float32), torch.tensor(labels, dtype=torch.long), centroid, max_distance

# Example usage
import glob
train_path = glob.glob("../input/3dmeshs/data/train/*.ply")
test_path = glob.glob("../input/3dmeshs/data/val/*.ply")
train_dataset = PointCloudDataset(train_path)
test_dataset = PointCloudDataset(test_path)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=15, shuffle=True)
print(len(train_dataset))
print(train_dataset.__getitem__(42))

In [ ]:
print(type(train_dataset))
train_dataset.__getitem__(0)

Training

In [ ]:
from sklearn.metrics import f1_score

# Initialize model, loss, and optimizer, and train the model
model = SimplePointNet(num_classes=8)  # Adjust num_classes as needed
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 75
for epoch in range(num_epochs):
    model.train()
    correct_train = 0
    total_train = 0
    all_train_preds = []
    all_train_labels = []

    for batch_idx, (points, labels, centroid, max_distance) in enumerate(train_loader):
        points = points.transpose(1, 2)  # (B, 3, N)
        labels = labels.long()

        outputs = model(points)  # (B, num_classes, N)
        outputs = outputs.permute(0, 2, 1)  # (B, N, num_classes)

        loss = criterion(outputs.reshape(-1, model.num_classes), labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy and F1 accumulation
        preds = outputs.argmax(dim=2)  # (B, N)
        correct_train += (preds == labels).sum().item()
        total_train += labels.numel()

        all_train_preds.extend(preds.cpu().numpy().flatten())
        all_train_labels.extend(labels.cpu().numpy().flatten())

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

    train_acc = 100 * correct_train / total_train
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    # Test evaluation
    model.eval()
    correct_test = 0
    total_test = 0
    all_test_preds = []
    all_test_labels = []

    with torch.no_grad():
        for points, labels, centroid, max_distance in test_loader:
            points = points.transpose(1, 2)
            labels = labels.long()

            outputs = model(points)
            outputs = outputs.permute(0, 2, 1)
            preds = outputs.argmax(dim=2)

            correct_test += (preds == labels).sum().item()
            total_test += labels.numel()

            all_test_preds.extend(preds.cpu().numpy().flatten())
            all_test_labels.extend(labels.cpu().numpy().flatten())

    test_acc = 100 * correct_test / total_test
    test_f1 = f1_score(all_test_labels, all_test_preds, average='weighted', zero_division=0)

    print(f"Epoch [{epoch+1}/{num_epochs}] — Train Acc: {train_acc:.2f}%, F1: {train_f1:.4f} | Test Acc: {test_acc:.2f}%, F1: {test_f1:.4f}")


In [ ]:
# Save model
# torch.save(model.state_dict(), "model_normalize.pth")

In [ ]:
# Load model
model = SimplePointNet(num_classes=8)
model.load_state_dict(torch.load("model_normalize.pth"))

## Evaluation and testing

### Point Cloud Metrics

In [ ]:
# point cloud accuracy
import os
from sklearn.metrics import f1_score
import sys

# Set model to evaluation mode
model.eval()

# Directory containing validation files
val_dir = "../input/3dmeshs/data/val"
val_files = glob.glob(os.path.join(val_dir, "*.ply"))

# Initialize lists to store metrics
accuracies = []
f1_scores = []
samples_points = {}
samples_predicted_labels = {}
# for mesh metrics
centroids = {}
max_distances = {}

# flush the output
sys.stdout.flush()
# Iterate over all validation files
for file_path in val_files:
    # Load data
    points, true_labels = load_and_prepare_data(file_path, num_points=10000)
    samples_points.update({os.path.basename(file_path): points})
    points, centroid, max_distance = normalize_point_cloud(points)
    centroids.update({os.path.basename(file_path): centroid})
    max_distances.update({os.path.basename(file_path): max_distance})
    # Prepare input tensor

    points = torch.tensor(points, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    points = points.transpose(1, 2)  # Shape: (1, 3, num_points)

    # Predict labels
    with torch.no_grad():
        outputs = model(points)
        predicted_labels = torch.argmax(outputs, dim=1)  # Shape: (1, num_points)
        samples_predicted_labels.update({os.path.basename(file_path): predicted_labels})

    # Convert ground truth labels to tensor
    true_labels = torch.tensor(true_labels, dtype=torch.long)  # Shape: (num_points,)

    # Calculate accuracy
    correct = (predicted_labels == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = correct / total
    accuracies.append(accuracy)

    # Calculate F1 score
    f1 = f1_score(true_labels.numpy(), predicted_labels.squeeze(0).numpy(), average="weighted")
    f1_scores.append(f1)

    print(f"File: {os.path.basename(file_path)} | Accuracy: {accuracy * 100:.2f}% | F1 Score: {f1:.4f}")

# Calculate average accuracy and F1 score
avg_accuracy = np.mean(accuracies)
avg_f1_score = np.mean(f1_scores)

print(f"\nAverage Accuracy: {avg_accuracy * 100:.2f}%")
print(f"Average F1 Score: {avg_f1_score:.4f}")

### Mesh Metrics

In [ ]:
import os
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.neighbors import NearestNeighbors
from plyfile import PlyData
import glob

# Directories
val_dir = "data/val"
results_dir = "results/predicted labels"
os.makedirs(results_dir, exist_ok=True)  # Create results directory if it doesn't exist

# Get all PLY files in the validation directory
val_files = glob.glob(os.path.join(val_dir, "*.ply"))

# Initialize lists to store metrics
all_accuracies = []
all_f1_scores = []

all_true_labels = []
all_predicted_labels = []

# Iterate over all validation files
for file_path in val_files:
    # Load the PLY file
    ply_data = PlyData.read(file_path)

    # Extract vertices and true labels
    vertices = np.vstack([
        ply_data['vertex']['x'],
        ply_data['vertex']['y'],
        ply_data['vertex']['z']
    ]).T  # Shape: (N, 3)

    vertex_labels = np.array(ply_data['vertex']['label'])  # Shape: (N,)
    all_true_labels.append(vertex_labels) # for classification report

    points = samples_points[os.path.basename(file_path)]
    points = denormalize_point_cloud(points, centroids[os.path.basename(file_path)], max_distances[os.path.basename(file_path)])
    # Find nearest neighbors between mesh vertices and point cloud
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(points)
    distances, indices = nbrs.kneighbors(vertices)

    # Assign labels to the mesh vertices
    predicted_vertex_labels = samples_predicted_labels[os.path.basename(file_path)].flatten()[indices.flatten()]
    all_predicted_labels.append(predicted_vertex_labels) # for classification report

    # Save predicted labels to a file
    sample_name = os.path.splitext(os.path.basename(file_path))[0]
    output_file = os.path.join(results_dir, f"{sample_name}_labels.txt")
    np.savetxt(output_file, predicted_vertex_labels, fmt='%d')

    # Calculate accuracy and F1 score for this sample
    accuracy = accuracy_score(vertex_labels, predicted_vertex_labels)
    f1 = f1_score(vertex_labels, predicted_vertex_labels, average="weighted")

    # Append to lists
    all_accuracies.append(accuracy)
    all_f1_scores.append(f1)
    print(f"Sample: {sample_name} | Accuracy: {accuracy * 100:.2f}% | F1 Score: {f1:.4f}")

# Calculate average metrics
avg_accuracy = np.mean(all_accuracies)
avg_f1_score = np.mean(all_f1_scores)

# Print overall metrics
print("\nOverall Metrics:")
print(f"Average Accuracy: {avg_accuracy * 100:.2f}%")
print(f"Average F1 Score: {avg_f1_score:.4f}")

In [ ]:
from sklearn.metrics import classification_report
class_names = ["head", "neck", "torso", "left_arm", "right_arm", "hip", "legs"]
print(classification_report(np.concatenate(all_true_labels), np.concatenate(all_predicted_labels), target_names=class_names))

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(np.concatenate(all_true_labels), np.concatenate(all_predicted_labels))

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

## Visualizing Predictions and Ground Truth

In [ ]:
# generates colored output for either original mesh or predicted mesh
import numpy as np
from plyfile import PlyData, PlyElement
def generate_colored_output(mesh_path, predicted_labels_path = None):
    sample_name = mesh_path.split("/")[-1].split(".")[0]

    # Load the original mesh PLY file
    ply_data = PlyData.read(mesh_path)

    # Extract vertices and original labels
    vertices = np.vstack([
        ply_data['vertex']['x'],
        ply_data['vertex']['y'],
        ply_data['vertex']['z']
    ]).T  # Shape: (N, 3)

    if predicted_labels_path is not None:
        # predicted_nameofthesameple_colored.ply
        vertex_labels = np.loadtxt(predicted_labels_path, dtype=int)
        output_name = f"predicted_{sample_name}_colored.ply"
        output_dir = "results/predicted colored"
        output_path = os.path.join(output_dir, output_name)
    else:
        # nameofthesample_colored.ply
        vertex_labels = np.array(ply_data['vertex']['label'])  # Shape: (N,)
        output_name = f"{sample_name}_colored.ply"
        output_dir = "results/original colored"
        output_path = os.path.join(output_dir, output_name)


    # Define class colors (RGB format, range 0-255)
    class_colors = {
        1: (255, 0, 0),    # Red
        2: (0, 255, 0),    # Green
        3: (0, 0, 255),    # Blue
        4: (255, 255, 0),  # Yellow
        5: (255, 165, 0),  # Orange
        6: (128, 0, 128),  # Purple
        7: (0, 255, 255),  # Cyan
    }

    # Assign colors based on predicted labels
    colors = np.array([class_colors[label] for label in vertex_labels], dtype=np.uint8)

    # Create a new structured array for the vertex data
    vertex_data = np.empty(len(vertices), dtype=[
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),  # Vertex positions
        ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),  # Vertex colors
        ('label', 'i4')  # Predicted labels
    ])

    # Fill vertex data with positions, colors, and predicted labels
    vertex_data['x'] = vertices[:, 0]
    vertex_data['y'] = vertices[:, 1]
    vertex_data['z'] = vertices[:, 2]
    vertex_data['red'] = colors[:, 0]
    vertex_data['green'] = colors[:, 1]
    vertex_data['blue'] = colors[:, 2]
    vertex_data['label'] = vertex_labels

    # Extract faces from the original mesh (if it's a mesh)
    if 'face' in ply_data:
        faces = ply_data['face']['vertex_indices']
        face_data = np.array([(tuple(face),) for face in faces], dtype=[('vertex_indices', 'i4', (3,))])
    else:
        face_data = None  # No faces (point cloud)

    # Save the updated mesh with new colors and labels
    new_ply_vertices = PlyElement.describe(vertex_data, 'vertex')
    if face_data is not None:
        new_ply_faces = PlyElement.describe(face_data, 'face')
        PlyData([new_ply_vertices, new_ply_faces]).write(output_path)
    else:
        PlyData([new_ply_vertices]).write(output_path)

    print(f"Updated mesh saved as '{output_name}' in the '{output_dir}' directory")

In [ ]:
# generate colored output for either original mesh or predicted mesh
import os

mesh_dir = "data/val"
predicted_labels_dir = "results/predicted labels"
for file in os.listdir(mesh_dir):
    if file.endswith(".ply"):
        file_path = os.path.join(mesh_dir, file)
        predicted_labels_file = os.path.join(predicted_labels_dir, f"{os.path.splitext(file)[0]}_labels.txt")
        generate_colored_output(file_path, predicted_labels_file)
        generate_colored_output(file_path)
